<a href="https://colab.research.google.com/github/vyster/StockMArket/blob/master/25years.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv('Sensex25years.csv', parse_dates=['Date'])
df['Change %'] = df['Change %'].str.replace('%', '').astype(float)

# Filter days with < -1% change
dip_days = df[df['Change %'] < -1].copy()
dip_days['Year'] = dip_days['Date'].dt.year

# Create annual summary table
annual_summary = (dip_days.groupby('Year')
                  .agg(Total_Dip_Days=('Date', 'count'),
                       Dip_Dates=('Date', lambda x: x.dt.strftime('%Y-%m-%d').tolist()))
                  .reset_index())

print("Annual Market Dip Summary (Change < -1%):")
print(annual_summary.to_string(index=False))

# Investment strategy simulation
def calculate_returns(data, annual_investment, strategy):
    portfolio = {
        'units': 0,
        'total_invested': 0,
        'transactions': []
    }

    if strategy == 'monthly':
        monthly_investment = annual_investment / 12
        for year in data['Year'].unique():
            year_data = data[data['Year'] == year]
            for month in range(1, 13):
                investment_date = pd.Timestamp(f'{year}-{month}-01')
                price = df[df['Date'] == investment_date]['Price'].values
                if len(price) > 0:
                    price = price[0]
                    units = monthly_investment / price
                    portfolio['units'] += units
                    portfolio['total_invested'] += monthly_investment
                    portfolio['transactions'].append({
                        'date': investment_date.strftime('%Y-%m-%d'),
                        'amount': monthly_investment,
                        'units': units
                    })

    elif strategy == 'dip_days':
        for year in annual_summary['Year']:
            year_dips = dip_days[dip_days['Year'] == year]
            if not year_dips.empty:
                daily_investment = annual_investment / len(year_dips)
                for _, row in year_dips.iterrows():
                    units = daily_investment / row['Price']
                    portfolio['units'] += units
                    portfolio['total_invested'] += daily_investment
                    portfolio['transactions'].append({
                        'date': row['Date'].strftime('%Y-%m-%d'),
                        'amount': daily_investment,
                        'units': units
                    })

    current_value = portfolio['units'] * df.iloc[-1]['Price']
    return portfolio | {'current_value': current_value}

# Execute both strategies
monthly_strategy = calculate_returns(df, 120000, 'monthly')
dipday_strategy = calculate_returns(df, 120000, 'dip_days')

# Results comparison
comparison_table = pd.DataFrame({
    'Metric': ['Total Invested', 'Total Units', 'Current Value'],
    'Monthly SIP': [
        monthly_strategy['total_invested'],
        monthly_strategy['units'],
        monthly_strategy['current_value']
    ],
    'Dip Day Investing': [
        dipday_strategy['total_invested'],
        dipday_strategy['units'],
        dipday_strategy['current_value']
    ]
})

print("\nStrategy Comparison:")
print(comparison_table.to_string(index=False))

FileNotFoundError: [Errno 2] No such file or directory: 'Sensex25years.csv'

# New Section